## Задача движения тела с приложенной силой

In [3]:
%load_ext autoreload
%autoreload 2

import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Idx, Sum, Function, Matrix, Add, Eq
from sympy import diff, symbols, solve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi

# syspath trick
import sys
sys.path.append('../')

from tatarinov.utils.jupyter import display_list, display_obj
from tatarinov.problems.body_with_F.variables import *
from tatarinov.core.tatarinov import TatarinovSystem

from tatarinov.database.singleton import db
db.set_root('../data/body_with_F')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
Lval = Eq(L, m*(Derivative(x,t)**2 + Derivative(y,t)**2)/2 + J[s]*(Derivative(alpha, t)**2)/2)
# Lval
# db.save(Lval, 'L', description='Лагранжиан')

In [9]:
constraints = []
constraints += [Eq(Derivative(x,t), cos(alpha)*nu1 - sin(alpha)*nu2)]
constraints += [Eq(Derivative(y,t), sin(alpha)*nu1 + cos(alpha)*nu2)]
constraints += [Eq(Derivative(alpha,t), nu3)]
display_list(constraints)

Eq(Derivative(x(t), t), nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t)))

Eq(Derivative(y(t), t), nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t)))

Eq(Derivative(alpha(t), t), nu3(t))

####  Создадим объект для механической системы с уравнениями Татаринова 

In [10]:
TS = TatarinovSystem()

#### Зададим всё что нужно для составления уравнений  

In [11]:
%%time
TS.set_L(simplify(Lval))

CPU times: user 600 ms, sys: 0 ns, total: 600 ms
Wall time: 599 ms


In [12]:
TS.set_constraints(constraints)

In [13]:
%%time
L_star = TS.set_L_star(TS.sub_constraints(Lval))
display(TS.L_star)

Eq(L, m*(nu1(t)**2 + nu2(t)**2)/2 + nu3(t)**2*J[s]/2)

CPU times: user 695 ms, sys: 0 ns, total: 695 ms
Wall time: 694 ms


In [14]:
TS.set_p([p[1], p[2], p[3]])
TS.set_q([x, y, alpha])

In [15]:
TS.set_omega_equations(omegas=[omega[1], omega[2], omega[3]],
                       equations=[nu1, nu2, nu3])
# display_list(TS.omega_equations)

In [16]:
TS.set_v_equations(vs=[v[1], v[2], v[3]],
                  equations=[Derivative(x,t), Derivative(y,t), Derivative(alpha,t)])
# display_list(TS.v_equations)

#### В данном случае сила прикладывается

In [17]:
F = W*e['xi'] + T*e['eta']
TS.set_F(F)
# display(e['xi'])
# display(e['eta'])
# display(F)

In [18]:
TS.create_P()
display_list(TS.P)

_dubug output:_ 

Eq(Sum(P[k]*omega[k], (k, 1, 3)), Sum(p[k]*v[k], (k, 1, 3)))

---

_dubug output:_ 

Eq(nu1(t)*P[1] + nu2(t)*P[2] + nu3(t)*P[3], (nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t)))*p[2] + (nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t)))*p[1] + nu3(t)*p[3])

---

Eq(P[1], sin(alpha(t))*p[2] + cos(alpha(t))*p[1])

Eq(P[2], -sin(alpha(t))*p[1] + cos(alpha(t))*p[2])

Eq(P[3], p[3])

In [19]:
TS.create_Q(r_p=r['p'])
display_list(TS.Q)

-T*sin(alpha(t)) + W*cos(alpha(t))

T*cos(alpha(t)) + W*sin(alpha(t))

(-T*sin(alpha(t)) + W*cos(alpha(t)))*(-eta*cos(alpha(t)) - xi*sin(alpha(t))) + (T*cos(alpha(t)) + W*sin(alpha(t)))*(-eta*sin(alpha(t)) + xi*cos(alpha(t)))

In [20]:
for i in range(3):
    TS.tatarinov_equation(i)

_dubug output:_ 

__Уравнение #0__

---

_dubug output:_ 

__Левая часть уравнения__

...

---

_dubug output:_ 

__Правая часть уравнения__

T*xi - W*eta - (-sin(alpha(t))*p[1] + cos(alpha(t))*p[2])*nu1(t) - (-sin(alpha(t))*p[2] - cos(alpha(t))*p[1])*nu2(t)

---

In [23]:
display_list(TS.tatarinov_equations)

Eq(m*Derivative(nu1(t), t), W + m*nu2(t)*nu3(t))

Eq(m*Derivative(nu2(t), t), T - m*nu1(t)*nu3(t))

Eq(T*xi - W*eta, Derivative(nu3(t), t)*J[s])

In [27]:
db.save(Matrix(TS.tatarinov_equations), 'TEqs', description='Уравнения Татаринова')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/body_with_F
{'description': 'Уравнения Татаринова'}
Уравнения Татаринова


In [30]:
TS.tatarinov_equations

[Eq(m*Derivative(nu1(t), t), W + m*nu2(t)*nu3(t)),
 Eq(m*Derivative(nu2(t), t), T - m*nu1(t)*nu3(t)),
 Eq(T*xi - W*eta, Derivative(nu3(t), t)*J[s])]